## Thu Cao
MNIST Handwritten Dataset

data source: http://yann.lecun.com/exdb/mnist/

In [1]:
import scipy.io
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import cv2
import os
import glob
import skimage
from scipy import io as spio
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter

sn.set()

from sklearn.svm import SVC # SVC
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.nasnet import NASNetMobile # NASNetMobile
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
import keras
import warnings
warnings.filterwarnings("ignore")
import os

Using TensorFlow backend.



# Loading The Dataset

In [2]:
train = pd.read_csv("mnist_train.csv")
test = pd.read_csv("mnist_test.csv")
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = train.iloc[:, 1:].values
y = train.iloc[:, 0].values
X_test = test.iloc[:,1:].values
y_test = test.iloc[:, 0].values

In [5]:
X = X.reshape(X.shape[0], 28, 28)
X_test = X_test.reshape(X_test.shape[0], 28, 28)

In [6]:
X.shape, y.shape, X_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

## Preprocessing the Data
Creating three different functions:<br>
- **resize** - This is created to resize the images in the given size
- **resize_images** - This is resize the given array of images
- **to_RGB** - This is used to convert the Black & White image into RGB image

In [7]:
def resize(image,size):
    image2 = np.zeros(size)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image2[i+2,j+2] = image[i,j]
    return image2

def resize_images(array,size):
    
    for image in array:
        x=[]
        image2 = np.zeros(size)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                image2[i+2,j+2] = image[i,j]
            x.append(image2)
            
        finall = np.array(x)
        
    return finall

def to_RGB(image):
    shape = image.shape
    dim = np.zeros(shape)
    rr = np.stack((image,dim,dim),axis=2)
    return rr

In [8]:
x=[]
for image in X:
    x.append(resize(image,(32,32)))
train = np.array(x)

x=[]
for image in train:
    rr = to_RGB(image)
    x.append(rr)
train = np.array(x)

X = train/255.0

In [9]:
del train

In [10]:
x=[]
for image in X_test:
    x.append(resize(image,(32,32)))
test = np.array(x)

x=[]
for image in test:
    rr = to_RGB(image)
    x.append(rr)
test = np.array(x)

X_test = test/255.0

In [11]:
del test

One Hot Encoding the target variable using Keras in built function **to_categorical**

In [12]:
y = to_categorical(y, 10)
y_test = to_categorical(y_test, 10)

Function to plot training and validation accuracy and loss

In [13]:
def plot(history):
  accuracy = history.history['acc']
  val_accuracy = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = len(accuracy)

  plt.plot(range(epochs), accuracy, 'b', label='Training accuracy')
  plt.plot(range(epochs), val_accuracy, 'r', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(range(epochs), loss, 'b', label='Training Loss')
  plt.plot(range(epochs), val_loss, 'r', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

## VGG16

In [14]:
base_model_vgg16 = VGG16(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in base_model_vgg16.layers:
    layer.trainable=False

# Building Model

model = Sequential()
model.add(base_model_vgg16)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 14,848,586
Trainable params: 133,898
Non-trainable params: 14,714,688
_________________________________________________________________


In [15]:
len(model.layers)

4

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
import time
t0=time.time()
vgg16_history  = model.fit(X,y,epochs=20,validation_split = 0.2, batch_size=16 )
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
47984/48000 [============================>.] - ETA: 0s - loss: 0.3118 - acc: 0.9026

#model.evaluate(X_test,y_test)

In [ ]:
t1=time.time()
model.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()

In [ ]:
plot(vgg16_history)

# VGG19

We have changed the last fuly connected layer to have 10 neurons instead of the 4096 present in the architecture in order to classify images into ten different classes.

In [ ]:
base_model_vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in base_model_vgg19.layers:
    layer.trainable=False

# Building Model

model = Sequential()
model.add(base_model_vgg19)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.summary()

In [ ]:
len(model.layers)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
t0=time.time()
vgg19_history  = model.fit(X,y,epochs=20,validation_split = 0.2 )
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()



In [ ]:
t1=time.time()
model.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()


The Test set __Accuracy__ is __97.36%__ 

In [ ]:
plot(vgg19_history)

# ResNet-50

In [ ]:
resnet50_model = ResNet50(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in resnet50_model.layers:
    layer.trainable=False
    
resnet_50_model = Sequential()
resnet_50_model.add(resnet50_model)
resnet_50_model.add(Flatten())
resnet_50_model.add(Dense(256, activation='relu'))
resnet_50_model.add(Dense(10, activation='softmax'))
resnet_50_model.summary()

In [ ]:
len(resnet_50_model.layers)

In [ ]:
resnet_50_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
t0=time.time()
resnet_50_history  = resnet_50_model.fit(X,y,epochs=20,validation_split = 0.2)
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()

In [ ]:
t1=time.time()
resnet_50_model.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()



The Test Set __Accuracy__ is __86.76%__

In [ ]:
plot(resnet_50_history)

# ResNet-101

In [ ]:
resnet101_model = tensorflow.keras.applications.ResNet101(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in resnet101_model.layers:
    layer.trainable=False
    
resnet_101_model = Sequential()
resnet_101_model.add(resnet101_model)
resnet_101_model.add(Flatten())
resnet_101_model.add(Dense(64, activation='relu'))
resnet_101_model.add(Dense(10, activation='sigmoid'))
resnet_101_model.summary()

In [ ]:
len(resnet_101_model.layers)

In [ ]:
resnet_101_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
t0=time.time()
resnet_101_history  = resnet_101_model.fit(X,y,epochs=20,validation_split = 0.2)
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()


In [ ]:
t1=time.time()
resnet_101_model.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()





The Test Set  __Accuracy__  is __75.99%__

In [ ]:
plot(resnet_101_history)

# DenseNet 121

In [ ]:
base_model_densenet121 = DenseNet121(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in base_model_densenet121.layers:
    layer.trainable=False

# Building Model

model_densenet_121 = Sequential()
model_densenet_121.add(base_model_densenet121)
model_densenet_121.add(Flatten())
model_densenet_121.add(Dense(256, activation='relu'))
model_densenet_121.add(Dense(10, activation='sigmoid'))

model_densenet_121.summary()

In [ ]:
len(model_densenet_121.layers)

In [ ]:
model_densenet_121.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
t0=time.time()
densenet_121_history  = model_densenet_121.fit(X,y,epochs=20,validation_split =0.2 )
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()



In [ ]:
t1=time.time()
model_densenet_121.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()





The Test Set __Accuracy__ is __97.03%__

In [ ]:
plot(densenet_121_history)

### MobileNetv2

In [ ]:
base_model_mobilenetv2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(32, 32, 3))

# Freezing Layers
for layer in base_model_mobilenetv2.layers:
    layer.trainable=False

# Building Model

model_mobilenetv2 = Sequential()
model_mobilenetv2.add(base_model_mobilenetv2)
model_mobilenetv2.add(Flatten())
model_mobilenetv2.add(Dense(256, activation='relu'))
model_mobilenetv2.add(Dense(10, activation='sigmoid'))

model_mobilenetv2.summary()

In [ ]:
len(mdoel_mobilenetv2.layers)

In [ ]:
model_mobilenetv2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
t0=time.time()
mobilenetv2_history  = model_mobilenetv2.fit(X,y,epochs=20,validation_split = 0.2)
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()

In [ ]:
t1=time.time()
model_mobilenetv2.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()



The Test Set **Accuracy** is **75.44%**

In [ ]:
plot(mobilenetv2_history)

# AlexNet

In [ ]:
cnn_model = Sequential([
                                     
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(32,32,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),

    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation='softmax')  
])
cnn_model.summary()

In [ ]:
len(cnn_model.layers)

In [ ]:
cnn_model.compile(loss = 'categorical_crossentropy',optimizer='adam', metrics = ['acc'])

In [ ]:
t0=time.time()
cnn_history  = cnn_model.fit(X,y,epochs=20,validation_split = 0.2 )
print("Time taken to train the model:", round(time.time()-t0, 2), "seconds")
print("==========================================")
print()

In [ ]:
t1=time.time()
cnn_model.evaluate(X_test,y_test)
print("Time taken to predict test cases:", round(time.time()-t1, 2), "seconds")
print("==========================================")
print()





The Test Set **Accuracy** is **98.66%** 

In [ ]:
plot(cnn_history)